In [4]:
from dotenv import load_dotenv
import google.generativeai as genai
import textwrap
import json
import os

In [5]:
os.getcwd()

'c:\\Users\\laksh\\OneDrive\\Desktop\\WebDev\\FFCS\\tests'

In [22]:
load_dotenv()

genai.configure(api_key=os.getenv("genai"))
myfile = genai.upload_file("test4.jpg")

In [55]:
slot = genai.protos.Schema(
    type = genai.protos.Type.OBJECT,
    properties = {
        'faculty':genai.protos.Schema(type=genai.protos.Type.STRING),
        'slot':genai.protos.Schema(type=genai.protos.Type.STRING),
        'venue':genai.protos.Schema(type=genai.protos.Type.STRING),
        'course_type':genai.protos.Schema(type=genai.protos.Type.STRING),
    },
    required=['faculty','slot','venue','course_type']
)

slots = genai.protos.Schema(
    type=genai.protos.Type.ARRAY,
    items=slot
)

course = genai.protos.Schema(
    type=genai.protos.Type.OBJECT,
    properties={
        'course_list':genai.protos.Schema(type=genai.protos.Type.STRING),
        'semeseter':genai.protos.Schema(type=genai.protos.Type.STRING),
        'curriculum_category':genai.protos.Schema(type=genai.protos.Type.STRING),
    },
)

In [57]:
extract = genai.protos.FunctionDeclaration(
    name="add_to_database",
    description=textwrap.dedent("""\
        Adds entities to the database.
        """),
    parameters=genai.protos.Schema(
        type=genai.protos.Type.OBJECT,
        properties={
            'course':course,
            'slots': slots
        }
    )
)

In [58]:
model = genai.GenerativeModel("gemini-1.5-pro-latest",tools=[extract])
result = model.generate_content([
    myfile,
    "Extract information from the image"],
    tool_config={'function_calling_config':'ANY'}
    )

In [59]:
fc = result.candidates[0].content.parts[0].function_call
data = type(fc).to_dict(fc)
print(json.dumps(data["args"],indent=4))

{
    "slots": [
        {
            "course_type": "ETH",
            "faculty": "NIVITHA K",
            "slot": "PRP323",
            "venue": "A2"
        },
        {
            "faculty": "THANGARAMYA K",
            "course_type": "ETH",
            "slot": "PRP414",
            "venue": "A2"
        },
        {
            "course_type": "ETH",
            "faculty": "THANGARAMYA K",
            "slot": "PRP414",
            "venue": "A1"
        },
        {
            "course_type": "ETH",
            "slot": "PRP323",
            "faculty": "NIVITHA K",
            "venue": "A1"
        },
        {
            "faculty": "SUNIL KUMAR",
            "slot": "PRP411",
            "course_type": "ETH",
            "venue": "A2"
        },
        {
            "course_type": "ETH",
            "faculty": "S M FAROOQ",
            "slot": "PRP575",
            "venue": "A2"
        },
        {
            "faculty": "SUNIL KUMAR",
            "course_type": "ETH",
        